# INF-616 - Tarefa 1

Professor: Jacques Wainer -- wainer@ic.unicamp.br
Monitor: Lucas David -- ra188972@students.ic.unicamp.br

Instituto de Computação - Unicamp  
2018

## Classificação binária (decisão)

In [ ]:
from __future__ import print_function

import numpy as np
import matplotlib

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
seed = 42
test_size = 0.25

np.random.seed(seed)
dataset = load_breast_cancer()


def train_test_split(x, y, test_fraction=1 / 3):
    """Separa um conjunto de `len(x)` amostras em dois de
    tamanhos `len(x) * (1 - test_fraction)` e `len(x) * test_fraction`.
    """
    indices = np.arange(len(x))
    np.random.shuffle(indices)
    
    train_size = int(len(x) * (1 - test_fraction))
    train_samples = indices[:train_size]
    test_samples = indices[train_size:]
    
    return (x[train_samples], y[train_samples]), (x[test_samples], y[test_samples])


(x_train, y_train), (x_test, y_test) = train_test_split(dataset.data,
                                                        dataset.target,
                                                        test_size)

print('Conjunto "Breast Cancer Wisconsin (Diagnostic)""',
      '================================================\n',
      'Contém descrições dos núcleos das células aspiradas de tecido ',
      'mamário humano, classificadas em malignas e benignas.\n',
      sep='\n')

print('Amostras no conjunto original: ', len(dataset.data))
print('Amostras no conjunto de treino:', len(x_train))
print('Amostras no conjunto de teste: ', len(x_test))

### Inicialize e treine um classificador `SVC` e um `LogisticRegression`

Utilize as classes `sklearn.svm.SVC` e `sklearn.linear.LogisticRegression`

In [ ]:
SVC_C            = 100
SVC_GAMMA        = 0.001
SVC_RANDOM_STATE = 13
LR_C             = 1000
LR_RANDOM_STATE  = 24

# Inicialize e treine os estimadores aqui:
# ...
# svc = ...
# lr = ...

estimators = [svc, lr]

### Qual dos dois estimadores apresenta maior acurácia sobre o conjunto WDBC teste?

In [ ]:
for e in estimators:
    print('~> Teste do estimator', type(e).__name__)
    p_train = e.predict(x_train)
    
    print('   exemplo de rótulos de treino verdadeiros:', y_train[:10], '...')
    print('   exemplo de rótulos de treino preditos:', p_train[:10], '...')

    # Resposta:
    # ...

### O *Support Vector Machine* Classifier consegue distinguir ambas as classes de forma satisfatória?

Utilize a matriz de confusão para descobrir se o SVC consegue efetivamente distinguir amostras benignas de malignas.

In [ ]:
p_test = svc.predict(x_test)

# Resposta:
# ...

### Calcule manualmente (sem o auxílio do `scikit.metrics`) as métricas acurácia balanceada, precision, *recall* e *F1*

In [ ]:
for e in estimators:
    print('~> Estimador', type(e).__name__)
    p_test = e.predict(x_test)

    # Resposta:
    # a = calculo da acurácia...
    # p = calculo da precisao...
    # r = calculo do recall...
    # f1 = calculo do f1...
    
    print('   acuracia balanceada:', a)
    print('   precision:', p)
    print('      recall:', r)
    print('          f1:', f1)

### Recalcule as medidas anteriores e a curva ROC AUC com o auxílio do scikit-learn

In [ ]:
print('~> Estimador', type(svc).__name__)
p_test = svc.predict(x_test)
s_test = svc.decision_function(x_test)

# Resposta:
# ...

In [ ]:
def display_roc_curve(y, s):
    """Exibe a curva ROC AUC em um grafico.
    Originalmente implementado em: http://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
    """
    fpr, tpr, _ = metrics.roc_curve(y, s)
    roc_auc = metrics.auc(fpr, tpr)

    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, color='orange', lw=4, alpha=.6, label='ROC curve (area: %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], color='black', lw=4, alpha=0.6, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc="lower right")
    [spine.set_visible(False) for spine in plt.gca().spines.values()]


display_roc_curve(y_test, s_test)

In [ ]:
def display_predictions(y, s):
    """Exibe a distribuicao das predicoes de ambas classes + e -.
    """
    # Originally implemented at: https://towardsdatascience.com/histograms-and-density-plots-in-python-f6bda88f5ac0

    plt.figure(figsize=(12, 6))

    sns.distplot(s[~y.astype(bool)], hist=False, kde=True, kde_kws={'shade': True, 'linewidth': 3}, label='Benign')
    sns.distplot(s[y.astype(bool)], hist=False, kde=True, kde_kws={'shade': True, 'linewidth': 3}, label='Malign')

    plt.title('Prediction Distributions')
    plt.xlabel('predicted value')
    plt.ylabel('density')
    plt.legend(prop={'size': 16}, loc='upper right')
    [spine.set_visible(False) for spine in plt.gca().spines.values()]
    

display_predictions(y_test, s_test)